# Is het verdamping?

In dit experiment is er een verwarmingselement in een met water gevulde maatbeker gestopt. Elke minuut is de temperatuur van het water gemeten. Deze metingen zijn opgeslagen in [tempmetingen.csv](tempmetingen.csv).

Verder is gegeven dat: 
$m_{maatbeker} = 820.8 \mathrm{gr}$  
$m_{maatbeker + water, start} = 1292.9 \mathrm{gr}$  
$m_{maatbeker + water, eind} = 1274.9 \mathrm{gr}$

Het moge duidelijk zijn dat er water is 'verdwenen'. Ook is de eindtemperatuur van het water niet gelijk aan een proces waarbij verdamping en eventuele warmteverliezen niet meegenomen worden. 

```{exercise}
Onderzoek het proces door:
1. De plot van de metingen te maken, inclusief een verwachte trendlijn voor de temperatuur van het water als al de toegevoerde warmte gaat zitten in het water.
2. Bepaal of de totale hoeveelheid toegevoegde energie correspondeert met de opwarming en verdamping van het water, geef daarbij de aannames die je doet.
3. Geef aanbevelingen voor een verbetering van het experiment om (nog) beter het thermodynamische proces te kunnen onderzoeken en beschrijven.
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# ============================================================================
# EXPERIMENT VERDAMPING - DATA INLADEN EN ANALYSE
# ============================================================================

# Gegeven waarden
m_beker = 820.8  # gram
m_beker_water_start = 1292.9  # gram
m_beker_water_eind = 1274.9  # gram

# Berekende waarden
m_water_start = m_beker_water_start - m_beker
m_water_eind = m_beker_water_eind - m_beker
m_water_verdampt = m_water_start - m_water_eind

print("="*70)
print("VERDAMPINGSEXPERIMENT - DATA ANALYSE")
print("="*70)
print(f"\nMassameting:")
print(f"  Initieel water: {m_water_start:.1f} g")
print(f"  Eindig water: {m_water_eind:.1f} g")
print(f"  Verdampt water: {m_water_verdampt:.1f} g ({(m_water_verdampt/m_water_start)*100:.1f}%)")

# Laad temperatuurmetingen
try:
    df = pd.read_csv('tempmetingen-733121424db047611d1756851296ade6.csv')
    print(f"\n✓ Data geladen: {len(df)} metingen")
    print(f"  Kolommen: {list(df.columns)}")
    print(f"\nEerste paar metingen:")
    print(df.head())
except FileNotFoundError:
    print("⚠️  Bestand niet gevonden. Zorg dat tempmetingen.csv in dezelfde map staat.")
    df = None

In [ ]:
# ============================================================================
# VRAAG 1: Plot meetgegevens met verwachte trendlijn
# ============================================================================

if df is not None:
    print("\n" + "="*70)
    print("VRAAG 1: TEMPERATUURMETING EN VERWACHTING")
    print("="*70)
    
    # Aannames voor verwachte trendlijn:
    # - Al de warmte gaat in het water (geen verdamping, geen verlies)
    # - Specifieke warmtecapaciteit water: c_water = 4186 J/(kg·K)
    
    c_water = 4186  # J/(kg·K)
    m_water_kg = m_water_start / 1000
    
    # Bepaal de tijd (minuten) en temperatuur
    tijd = np.arange(len(df))  # Minuten (0-indexed)
    T_meet = df.iloc[:, 0].values  # Eerste kolom is temperatuur
    T_start = T_meet[0]
    T_eind = T_meet[-1]
    
    print(f"\nGemeten temperaturen:")
    print(f"  Start: {T_start:.1f} °C")
    print(f"  Eind: {T_eind:.1f} °C")
    print(f"  Verschil: {T_eind - T_start:.1f} °C")
    print(f"  Tijd: {len(df)-1} minuten")
    
    # Verwachte lineaire stijging (bij constant vermogen P)
    # Voer aanname in: bepaal "equivalent" vermogen uit werkelijke stijging eerste periode
    # (voormaliger wij gaan ervan uit dat warmteverlies/verdamping later domineert)
    
    # Fit lineaire trendlijn
    z = np.polyfit(tijd, T_meet, 1)
    p = np.poly1d(z)
    T_fit = p(tijd)
    
    print(f"\nLineare fit (werkelijke data):")
    print(f"  Helling: {z[0]:.4f} °C/min")
    print(f"  Intercept: {z[1]:.2f} °C")
    
    # Visualisatie
    plt.figure(figsize=(12, 6))
    plt.scatter(tijd, T_meet, color='red', s=50, alpha=0.7, label='Meetpunten', zorder=3)
    plt.plot(tijd, T_fit, 'b-', linewidth=2, label=f'Lineaire fit: T = {z[0]:.4f}t + {z[1]:.2f}')
    plt.xlabel('Tijd (minuten)', fontsize=11, fontweight='bold')
    plt.ylabel('Temperatuur (°C)', fontsize=11, fontweight='bold')
    plt.title('Temperatuurverloop bij verdamping', fontsize=13, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Plot gemaakt")

# ============================================================================
# VRAAG 2: Energiebalans
# ============================================================================

print("\n" + "="*70)
print("VRAAG 2: ENERGIEBALANS")
print("="*70)

# Gegeven/aangenomen waarden
c_water = 4186  # J/(kg·K)
L_vap = 2.26e6  # J/kg - latente verdampingswarmte van water bij ~100°C
L_vap_gemiddeld = 2.4e6  # J/kg - meer realistisch voor 20-100°C

print("\nAannames:")
print(f"  Specifieke warmtecapaciteit water: {c_water} J/(kg·K)")
print(f"  Latente verdampingswarmte: {L_vap/1e6:.2f} MJ/kg")
print(f"  Water verdampt gelijkmatig over gehele experiment")
print(f"  Geen warmteverlies aan beker of omgeving (eerste benadering)")

# Energie voor opwarming
m_water_gemiddeld = (m_water_start + m_water_eind) / 2000  # in kg
delta_T = T_eind - T_start
Q_opwarming = m_water_gemiddeld * c_water * delta_T

print(f"\nWarmte voor opwarming:")
print(f"  ΔT = {delta_T:.1f} K")
print(f"  m_water (gem.) = {m_water_gemiddeld:.3f} kg")
print(f"  Q_opwarming = m·c·ΔT = {Q_opwarming:.0f} J = {Q_opwarming/1e3:.1f} kJ")

# Energie voor verdamping
m_verdampt_kg = m_water_verdampt / 1000
Q_verdamping = m_verdampt_kg * L_vap_gemiddeld

print(f"\nWarmte voor verdamping:")
print(f"  m_verdampt = {m_water_verdampt:.1f} g = {m_verdampt_kg:.4f} kg")
print(f"  L_vap = {L_vap_gemiddeld/1e6:.2f} MJ/kg")
print(f"  Q_verdamping = m·L_vap = {Q_verdamping:.0f} J = {Q_verdamping/1e3:.1f} kJ")

# Totale energie
Q_totaal_theoretisch = Q_opwarming + Q_verdamping

print(f"\nTotale energie (theoretisch):")
print(f"  Q_totaal = Q_opwarming + Q_verdamping")
print(f"  Q_totaal = {Q_opwarming:.0f} + {Q_verdamping:.0f}")
print(f"  Q_totaal = {Q_totaal_theoretisch:.0f} J = {Q_totaal_theoretisch/1e3:.1f} kJ")

# Bepaal gemiddeld vermogen
tijd_seconden = (len(df) - 1) * 60  # in seconden
P_gemiddeld = Q_totaal_theoretisch / tijd_seconden

print(f"\nGemiddeld vermogen:")
print(f"  Tijd: {len(df)-1} min = {tijd_seconden} s")
print(f"  P_gemiddeld = Q_totaal / t = {P_gemiddeld:.1f} W")

# ============================================================================
# VRAAG 3: Aanbevelingen
# ============================================================================

print("\n" + "="*70)
print("VRAAG 3: AANBEVELINGEN VOOR VERBETERING")
print("="*70)

aanbevelingen = """
1. WARMTEVERLIEZEN METEN
   • Meet omgevingstemperatuur
   • Isoleer de beker beter (thermische wollen doek)
   • Bepaal warmteverlies naar beker: Q_beker = m_beker·c_beker·ΔT
   
2. VERDAMPING METEN
   • Weeg water regelmatig (bijv. iedere 5 min) in plaats van alleen begin/eind
   • Dit geeft inzicht in het verdampingspatroon
   • Bepaal verdampingssnelheid als functie van temperatuur
   
3. NAUWKEURIGHEID
   • Gebruik nauwkeurigere thermometer (digitaal, ±0.1°C)
   • Stir water regelmatig (mengeffect is niet uniform)
   • Dezelfde verwarmingselement gebruiken (constant vermogen)
   • Meerdere herhalingen doen
   
4. DATA-ANALYSE VERBETERING
   • Bereken warmteverlies naar omgeving (Newton's wet van afkoeling)
   • Onderscheid fasen: opwarming → koken → verdamping
   • Bepaal exact moment van kookpunt (T = 100°C)
   • Analyseer verdampingssnelheid voor/na kookpunt
   
5. THERMODYNAMISCHE ANALYSE
   • Voeg energiemeter toe (direct vermogen meten)
   • Bepaal rendement: η = Q_totaal / Q_toegevoerd
   • Kwantificeer warmteverlies percentage
"""

print(aanbevelingen)

print("\n" + "="*70)